In [36]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
from holoviews import opts
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from kneed import KneeLocator
import warnings
warnings.filterwarnings("ignore")

In [37]:
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [38]:
df_market_data.columns

Index(['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y'],
      dtype='object')

In [39]:
df_market_data.index

Index(['bitcoin', 'ethereum', 'tether', 'ripple', 'bitcoin-cash',
       'binancecoin', 'chainlink', 'cardano', 'litecoin', 'bitcoin-cash-sv',
       'crypto-com-chain', 'usd-coin', 'eos', 'monero', 'tron', 'tezos', 'okb',
       'stellar', 'cosmos', 'cdai', 'neo', 'wrapped-bitcoin', 'leo-token',
       'huobi-token', 'nem', 'binance-usd', 'iota', 'vechain', 'zcash',
       'theta-token', 'dash', 'ethereum-classic', 'ethlend', 'maker', 'havven',
       'omisego', 'celsius-degree-token', 'ontology', 'ftx-token', 'true-usd',
       'digibyte'],
      dtype='object', name='coin_id')

In [40]:
# summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [41]:
# Plot data to see what's in your DataFrame
df_market_data.hvplot.line(rot=45, height=400, width=900)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [42]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
standardized = StandardScaler().fit_transform(df_market_data)
standardized[0:5]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402,
        -0.45825882, -0.29554614],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287,
        -0.49984776, -0.27031695]])

In [43]:
standardizedMarketData = pd.DataFrame(standardized, columns=df_market_data.columns, index=df_market_data.index)
standardizedMarketData

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351


---

### Find the Best Value for k Using the Original Data.

In [44]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [45]:
inertia = []

for i in k:
    kModel = KMeans(n_clusters=i, random_state=42)
    kModel.fit(standardizedMarketData)
    inertia.append(kModel.inertia_)

In [46]:
elbowData = pd.DataFrame({"inertia" : inertia})
elbowData.index = k
elbowData

,inertia
1,287.000000
2,195.820218
3,123.190482
4,79.022435
5,65.220364
6,52.829819
7,43.786435
8,37.527740
9,33.070926
10,28.989073


In [120]:
kn = KneeLocator(elbowData.index, elbowData["inertia"], curve='convex', direction='decreasing')
print("the optimal k value to use:", kn.knee)

vline = hv.VLine(kn.knee).opts(line_dash="dashed", color="#F3CA3C")

elbowGraph = elbowData.hvplot.line(label='Original Data') * vline
elbowGraph


the optimal k value to use: 4


:Overlay
   .Curve.Original_Data :Curve   [index]   (inertia)
   .VLine.I             :VLine   [x,y]

the best value for `k`: 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [48]:
# Initialize the K-Means model using the best value for k

In [49]:
cryptoKModel = KMeans(n_clusters=4, random_state=42)
cryptoKModel.fit(standardizedMarketData)
cryptoPredicted = cryptoKModel.predict(standardizedMarketData)
cryptoPredicted

array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [50]:
marketDataKModel = standardizedMarketData.copy()
marketDataKModel["K Means"] = cryptoPredicted
marketDataKModel.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,K Means
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [131]:
beforePCA = marketDataKModel.hvplot.scatter(x="price_change_percentage_24h", y="price_change_percentage_7d", 
                                by="K Means", hover_cols = ["coin_id"],
                                xlabel="% change in 24h", ylabel="% change in 7d",
                                title="Crypto Currencies Price Change", label='Original Data')
beforePCA

:NdOverlay   [K Means]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [52]:
# Create a PCA model instance and set `n_components=3`.
pcaModel = PCA(n_components=3)

In [53]:
marketPCA = pcaModel.fit_transform(standardizedMarketData)
marketPCA[:7]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527],
       [-0.51653377,  1.38837748,  0.80407131],
       [-0.45071134,  0.51769912,  2.84614316]])

In [54]:
# Retrieve the explained variance to determine how much information can be attributed to each principal component.
pcaModel.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

In [55]:
sum(pcaModel.explained_variance_ratio_)

0.8950316570309841

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89.5%

In [59]:
pcaGroupings = pd.DataFrame(marketPCA, columns=["PCA1", "PCA2", "PCA3"], index=df_market_data.index)
pcaGroupings.head(10)

,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715
binancecoin,-0.516534,1.388377,0.804071
chainlink,-0.450711,0.517699,2.846143
cardano,-0.345600,0.729439,1.478013
litecoin,-0.649468,0.432165,0.600303


---

### Find the Best Value for k Using the PCA Data

In [60]:
inertia = []

for i in k:
    kModel = KMeans(n_clusters=i, random_state=42)
    kModel.fit(pcaGroupings)
    inertia.append(kModel.inertia_)

pcaElbow = pd.DataFrame({"inertia" : inertia})
pcaElbow.index = k
pcaElbow

,inertia
1,256.874086
2,168.811896
3,93.774626
4,49.665497
5,37.878747
6,27.618972
7,21.182776
8,17.091637
9,13.667065
10,10.559358


In [126]:
kn = KneeLocator(pcaElbow.index, pcaElbow["inertia"], curve='convex', direction='decreasing')
print("the optimal k value to use:", kn.knee)

vline = hv.VLine(kn.knee).opts(line_dash="dashed", color="#F3CA3C")

pcaElbowGraph = pcaElbow.hvplot.line(label='PCA Data') * vline
pcaElbowGraph

the optimal k value to use: 4


:Overlay
   .Curve.PCA_Data :Curve   [index]   (inertia)
   .VLine.I        :VLine   [x,y]

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** no

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [62]:
# Initialize the K-Means model using the best value for k
PCAkModel = KMeans(n_clusters=4, random_state=42)
PCAkModel.fit(pcaGroupings)
pcaPredicted = PCAkModel.predict(pcaGroupings)
pcaPredicted

array([3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0, 0, 3, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 2, 3, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [65]:
pcaGroupingskMeans = pcaGroupings.copy()
pcaGroupingskMeans["PCA K Means"] = pcaPredicted
pcaGroupingskMeans #these are like the coordinates for each coin in a 3 dimentional space

,PCA1,PCA2,PCA3,PCA K Means
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,3
ethereum,-0.458261,0.458466,0.952877,3
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,3
binancecoin,-0.516534,1.388377,0.804071,3
chainlink,-0.450711,0.517699,2.846143,3
cardano,-0.345600,0.729439,1.478013,3
litecoin,-0.649468,0.432165,0.600303,3


In [132]:
pcaGroupingskMeans.hvplot.scatter(x="PCA1", y="PCA2", by="PCA K Means", hover_cols = ["coin_id"], title = "PCA components coordinates")

:NdOverlay   [PCA K Means]
   :Scatter   [PCA1]   (PCA2,coin_id)

In [68]:
marketCombined = pd.concat([standardizedMarketData, pcaGroupingskMeans], axis=1)
marketCombined.head(7)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,PCA1,PCA2,PCA3,PCA K Means
coin_id,,,,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,-0.600667,0.842760,0.461595,3
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,-0.458261,0.458466,0.952877,3
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,-0.433070,-0.168126,-0.641752,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,-1.157800,2.041209,1.859715,3
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533,-0.516534,1.388377,0.804071,3
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284,-0.450711,0.517699,2.846143,3


In [133]:
PCAgrouped = marketCombined.hvplot.scatter(x="price_change_percentage_24h", y="price_change_percentage_7d",
                              by="PCA K Means", hover_cols = ["coin_id"],
                                xlabel="% change in 24h", ylabel="% change in 7d",
                                title="Crypto Currencies Price Change After PCA", label='PCA Data')
PCAgrouped

:NdOverlay   [PCA K Means]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [127]:
# Composite plot to contrast the Elbow curves
(elbowGraph * pcaElbowGraph).opts(title="Elbow Curves Comparison")

:Overlay
   .Curve.Original_Data :Curve   [index]   (inertia)
   .VLine.I             :VLine   [x,y]
   .Curve.PCA_Data      :Curve   [index]   (inertia)
   .VLine.II            :VLine   [x,y]

In [134]:
beforePCA * PCAgrouped

:Overlay
   .NdOverlay.Original_Data :NdOverlay   [K Means]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.PCA_Data      :NdOverlay   [PCA K Means]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

In [135]:
# Composite plot to contrast the clusters
beforePCA + PCAgrouped

:Layout
   .NdOverlay.Original_Data :NdOverlay   [K Means]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.PCA_Data      :NdOverlay   [PCA K Means]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 

In [92]:
cluster1 = marketCombined[marketCombined["PCA K Means"]==0]
cluster2 = marketCombined[marketCombined["PCA K Means"]==1]
cluster3 = marketCombined[marketCombined["PCA K Means"]==2]
cluster4 = marketCombined[marketCombined["PCA K Means"]==3]
cluster4

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,PCA1,PCA2,PCA3,PCA K Means
coin_id,,,,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,-0.600667,0.842760,0.461595,3
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,-0.458261,0.458466,0.952877,3
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,-1.157800,2.041209,1.859715,3
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533,-0.516534,1.388377,0.804071,3
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284,-0.450711,0.517699,2.846143,3
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428,-0.345600,0.729439,1.478013,3
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351,-0.649468,0.432165,0.600303,3
monero,0.262723,1.792602,2.202665,1.437842,0.893865,-0.155893,-0.167644,0.060499,2.909404,1.498571,3
tezos,-0.151583,0.708196,0.258012,-0.602296,-0.956049,-0.449211,-0.168479,-0.796176,-0.494409,1.082812,3


In [102]:
print("CLUSTER 0: ","min %change in 24h=", cluster1.price_change_percentage_24h.min() , "max %change in 24h=", cluster1.price_change_percentage_24h.max())
print("CLUSTER 1: ","min %change in 24h=", cluster2.price_change_percentage_24h.min() , "max %change in 24h=", cluster2.price_change_percentage_24h.max())
print("CLUSTER 2: ","min %change in 24h=", cluster3.price_change_percentage_24h.min() , "max %change in 24h=", cluster3.price_change_percentage_24h.max())
print("CLUSTER 3: ","min %change in 24h=", cluster4.price_change_percentage_24h.min() , "max %change in 24h=", cluster4.price_change_percentage_24h.max())

CLUSTER 0:  min %change in 24h= -1.6121877504778772 max %change in 24h= 1.9198123431080822
CLUSTER 1:  min %change in 24h= 1.0455303437735397 max %change in 24h= 1.0455303437735397
CLUSTER 2:  min %change in 24h= -4.981041886343488 max %change in 24h= -4.981041886343488
CLUSTER 3:  min %change in 24h= -0.25597803962841204 max %change in 24h= 1.1930360799166912


In [103]:
print("CLUSTER 0: ","min %change in 7d=", cluster1.price_change_percentage_7d.min() , "max %change in 7d=", cluster1.price_change_percentage_7d.max())
print("CLUSTER 1: ","min %change in 7d=", cluster2.price_change_percentage_7d.min() , "max %change in 7d=", cluster2.price_change_percentage_7d.max())
print("CLUSTER 2: ","min %change in 7d=", cluster3.price_change_percentage_7d.min() , "max %change in 7d=", cluster3.price_change_percentage_7d.max())
print("CLUSTER 3: ","min %change in 7d=", cluster4.price_change_percentage_7d.min() , "max %change in 7d=", cluster4.price_change_percentage_7d.max())

CLUSTER 0:  min %change in 7d= -1.6820265588951313 max %change in 7d= 0.41404416555518786
CLUSTER 1:  min %change in 7d= -0.6183281619450216 max %change in 7d= -0.6183281619450216
CLUSTER 2:  min %change in 7d= -0.04517828983194763 max %change in 7d= -0.04517828983194763
CLUSTER 3:  min %change in 7d= 0.334296561906924 max %change in 7d= 2.5722509094400743


In [105]:
print("CLUSTER 0: ","avg %change in 7d=", cluster1.price_change_percentage_7d.mean() , "avg %change in 24h=", cluster1.price_change_percentage_24h.mean())
print("CLUSTER 3: ","avg %change in 7d=", cluster4.price_change_percentage_7d.mean() , "avg %change in 24h=", cluster4.price_change_percentage_24h.mean())

CLUSTER 0:  avg %change in 7d= -0.5699884128311491 avg %change in 24h= 0.03258562350382585
CLUSTER 3:  avg %change in 7d= 1.1910157834912964 avg %change in 24h= 0.23756041011311352
